In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ML'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive/
['settings.py', 'Youtube01-Psy.csv', 'Youtube03-LMFAO.csv', 'Youtube02-KatyPerry.csv', 'Youtube04-Eminem.csv', 'Youtube05-Shakira.csv', 'get_comment_data.py', 'vectorize_data.py', 'initial_eda.py', 'word2vecdat.csv', 'word2vecdat_nopunc.csv', 'data_with_vectors.csv', 'data_with_vectors2.csv', 'data.csv', 'word2vec_model', 'word2vecdat_googlemodel.csv', 'word2vec_vectorize', 'ML_final_model.ipynb']


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report

# Get Word2Vec data from csv file
dat = pd.read_csv(GOOGLE_DRIVE_PATH + '/word2vecdat.csv')
dat2 = pd.read_csv(GOOGLE_DRIVE_PATH + '/word2vecdat_nopunc.csv')

In [ ]:
dat = dat.iloc[:,1:]
dat2 = dat2.iloc[:,1:]

In [ ]:
print(dat.shape)
print(dat2.shape)

(1956, 101)
(1943, 101)


In [ ]:
X = dat.iloc[:,:-1]
Y = dat.iloc[:,-1]
X2 = dat2.iloc[:,:-1]
Y2 = dat2.iloc[:,-1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size = 0.2)

In [ ]:
svm = SVC()
svm.fit(X_train, Y_train)
Y_hat = svm.predict(X_test)
cm_svm = confusion_matrix(Y_test, Y_hat)
cr_svm = classification_report(Y_test, Y_hat, zero_division = 0)
print(cr_svm)

              precision    recall  f1-score   support

           0       0.53      0.99      0.69       180
           1       0.98      0.26      0.41       212

    accuracy                           0.60       392
   macro avg       0.76      0.63      0.55       392
weighted avg       0.78      0.60      0.54       392



In [ ]:
svm2 = SVC()
svm2.fit(X2_train, Y2_train)
Y2_hat = svm2.predict(X2_test)
cm_svm2 = confusion_matrix(Y2_test, Y2_hat)
cr_svm2 = classification_report(Y2_test, Y2_hat, zero_division = 0)
print(cr_svm2)

              precision    recall  f1-score   support

           0       0.62      0.27      0.38       182
           1       0.57      0.86      0.68       207

    accuracy                           0.58       389
   macro avg       0.60      0.56      0.53       389
weighted avg       0.59      0.58      0.54       389



In [ ]:
# Test Random Forest
rf = RandomForestClassifier(max_features =10, max_depth = 15)
rf.fit(X_train, Y_train)
Y_hat = rf.predict(X_test)
cm_rf = confusion_matrix(Y_test, Y_hat)
cr_rf = classification_report(Y_test, Y_hat, zero_division = 0)
print(cr_rf)

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       180
           1       0.89      0.85      0.87       212

    accuracy                           0.86       392
   macro avg       0.86      0.86      0.86       392
weighted avg       0.86      0.86      0.86       392



In [ ]:
# Test Random Forest
rf2 = RandomForestClassifier(max_features =10, max_depth = 15)
rf2.fit(X2_train, Y2_train)
Y2_hat = rf.predict(X2_test)
cm_rf2 = confusion_matrix(Y2_test, Y2_hat)
cr_rf2 = classification_report(Y2_test, Y2_hat, zero_division = 0)
print(cr_rf2)

              precision    recall  f1-score   support

           0       0.53      0.18      0.26       182
           1       0.54      0.86      0.67       207

    accuracy                           0.54       389
   macro avg       0.54      0.52      0.47       389
weighted avg       0.54      0.54      0.48       389



In [ ]:
# Hyperparameter optimization
import hyperopt
from hyperopt import hp, STATUS_OK
from hyperopt import Trials

def objective_linear(C):
  svc = SVC(C=C, kernel = 'linear')
  cv_results = cross_validate(svc, X, Y, cv = 5)
  acc = np.mean(cv_results['test_score'])
  return -acc


In [ ]:
linear_space = hp.uniform('C', 0, 200)

In [ ]:
poly_space = hp.choice('degree', [1,2,3,4,5,6,7,8,9])

In [ ]:
hyperopt.fmin(objective_linear,
              space = linear_space,
              algo = hyperopt.tpe.suggest,
              max_evals=30)
# Opt C = 99.49 acc 86.9

100%|██████████| 30/30 [00:37<00:00,  1.26s/it, best loss: -0.8686257111540268]


{'C': 99.49426020672412}

In [ ]:
-objective_linear(99.49426020672412)

0.8686257111540268

In [ ]:
C = 99.49426020672412
def objective_poly(params):
  degree = params
  svc = SVC(C=C, kernel = 'poly', degree = degree)
  cv_results = cross_validate(svc, X, Y, cv = 5)
  acc = np.mean(cv_results['test_score'])
  return -acc

hyperopt.fmin(objective_poly,
              space = poly_space,
              algo = hyperopt.tpe.suggest,
              max_evals=10)
# Best degree 3, acc 78.8

100%|██████████| 10/10 [00:14<00:00,  1.48s/it, best loss: -0.78786732084138]


{'degree': 3}

In [ ]:
def objective_rbf(params):
  C = params
  svc = SVC(C=C, kernel = 'rbf')
  cv_results = cross_validate(svc, X, Y, cv = 5)
  acc = np.mean(cv_results['test_score'])
  return -acc

hyperopt.fmin(objective_rbf,
              space = linear_space,
              algo = hyperopt.tpe.suggest,
              max_evals=30)
# Opt C 199.4, acc 82.4

100%|██████████| 30/30 [00:33<00:00,  1.10s/it, best loss: -0.8236324964768515]


{'C': 199.37913975452526}

In [ ]:
def objective_rf(params):
  rf = RandomForestClassifier(**params,n_jobs=-1)
  cv_results = cross_validate(rf, X, Y, cv = 5)
  acc = np.mean(cv_results['test_score'])
  return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()

space_rf = {
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
}

hyperopt.fmin(objective_rf,
              space = space_rf,
              algo = hyperopt.tpe.suggest,
              max_evals=30,
              trials = trials)
# opt crit 0, max_depth 13, n_est 5, acc 81.8

100%|██████████| 30/30 [09:09<00:00, 18.31s/it, best loss: -0.8175061328879378]


{'criterion': 0, 'max_depth': 13.0, 'n_estimators': 5}

In [ ]:
dat3 = pd.read_csv(GOOGLE_DRIVE_PATH + '/word2vecdat_googlemodel.csv')


In [ ]:
dat3 = dat3.iloc[:,1:]

In [ ]:
X3 = dat3.iloc[:,:-1]
Y3 = dat3.iloc[:,-1]
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y3, test_size = 0.2)

In [ ]:
svm3 = SVC()
svm3.fit(X3_train, Y3_train)
Y3_hat = svm3.predict(X3_test)
cm_svm3 = confusion_matrix(Y3_test, Y3_hat)
cr_svm3 = classification_report(Y3_test, Y3_hat, zero_division = 0)
print(cr_svm3)

              precision    recall  f1-score   support

           0       0.67      0.92      0.77       210
           1       0.83      0.47      0.60       182

    accuracy                           0.71       392
   macro avg       0.75      0.70      0.69       392
weighted avg       0.75      0.71      0.69       392



In [ ]:
# Test Random Forest
rf3 = RandomForestClassifier(max_features =10, max_depth = 15)
rf3.fit(X3_train, Y3_train)
Y3_hat = rf3.predict(X3_test)
cm_rf3 = confusion_matrix(Y3_test, Y3_hat)
cr_rf3 = classification_report(Y3_test, Y3_hat, zero_division = 0)
print(cr_rf3)

              precision    recall  f1-score   support

           0       0.87      0.84      0.85       210
           1       0.82      0.85      0.84       182

    accuracy                           0.84       392
   macro avg       0.84      0.84      0.84       392
weighted avg       0.85      0.84      0.84       392



In [ ]:
def objective_rf(params):
  rf = RandomForestClassifier(**params,n_jobs=-1)
  cv_results = cross_validate(rf, X3, Y3, cv = 5)
  acc = np.mean(cv_results['test_score'])
  return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()

space_rf = {
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
}

hyperopt.fmin(objective_rf,
              space = space_rf,
              algo = hyperopt.tpe.suggest,
              max_evals=30,
              trials = trials)

# Best crit 1, max_depth 12, n_est 5 acc 82.4

100%|██████████| 30/30 [21:25<00:00, 42.85s/it, best loss: -0.8236377159559476]


{'criterion': 1, 'max_depth': 12.0, 'n_estimators': 5}

In [ ]:
dat4 = pd.read_csv(GOOGLE_DRIVE_PATH + '/data2.csv')
X4 = dat4.iloc[:,2:]
Y4 = dat4.iloc[:,1]
X4_train, X4_test, Y4_train, Y4_test = train_test_split(X4, Y4, test_size = 0.2)

In [ ]:
X4

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.045250,0.066106,-0.021016,0.068848,0.072306,0.010498,0.016459,-0.109701,0.015055,0.146973,...,0.057729,0.062134,-0.068685,-0.152262,-0.066081,-0.010579,0.053955,-0.170898,0.047628,-0.021240
1,0.048737,0.009216,-0.009399,0.080811,-0.083801,-0.034180,0.026291,-0.034687,0.164825,0.130280,...,-0.059570,0.171143,-0.128784,0.019897,0.050453,-0.115326,-0.029175,-0.141220,0.018707,-0.067566
2,0.058167,-0.055298,0.011882,0.140516,-0.098748,0.029577,0.077148,-0.110189,0.129896,0.034749,...,0.037543,0.045926,-0.070150,-0.077189,0.013956,-0.106988,0.070014,-0.089084,0.009359,-0.028293
3,-0.054199,0.017090,-0.005280,0.332031,-0.250000,-0.013977,-0.150391,-0.265625,0.016479,0.382812,...,-0.024536,0.110840,-0.025146,-0.120605,0.052979,0.071289,0.000637,-0.365234,-0.137695,-0.128906
4,0.034403,-0.007044,-0.008789,0.095350,-0.112467,0.033915,0.100395,-0.088664,0.162679,0.012655,...,-0.039469,-0.002607,0.000092,-0.027120,-0.040609,-0.124654,0.050171,-0.110962,0.022995,-0.071025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,0.084863,-0.015234,0.057629,0.053003,-0.075073,0.024341,0.083807,-0.068057,0.094141,0.069580,...,0.040283,0.156250,-0.268945,-0.226581,-0.048291,-0.084766,-0.037695,-0.133694,-0.024683,0.029773
1788,0.006851,-0.023842,-0.029945,0.125061,-0.033142,-0.014618,0.063416,-0.177317,-0.018005,0.121292,...,-0.037994,0.179077,-0.093567,-0.049063,-0.050529,-0.037888,-0.021790,-0.089523,0.037231,-0.012176
1789,0.125000,0.018066,0.057617,0.002274,-0.064453,-0.050781,0.034424,-0.171875,0.083008,0.232422,...,0.269531,0.097656,-0.237305,0.134766,-0.149414,-0.183594,0.125000,-0.125977,-0.059082,0.010376
1790,-0.117676,-0.017212,0.113281,0.099731,-0.136078,-0.086182,-0.096924,-0.225098,-0.066162,0.125732,...,0.099854,0.130859,-0.152832,-0.119873,-0.120605,-0.138672,-0.142944,-0.160156,-0.065552,0.189941


In [ ]:
svm4 = SVC()
svm4.fit(X4_train, Y4_train)
Y4_hat = svm4.predict(X4_test)
cm_svm4 = confusion_matrix(Y4_test, Y4_hat)
cr_svm4 = classification_report(Y4_test, Y4_hat, zero_division = 0)
print(cr_svm4)

              precision    recall  f1-score   support

       False       0.85      0.96      0.90       166
        True       0.96      0.85      0.90       193

    accuracy                           0.90       359
   macro avg       0.90      0.91      0.90       359
weighted avg       0.91      0.90      0.90       359



In [ ]:
# Test Random Forest
rf4 = RandomForestClassifier(max_features =10, max_depth = 15)
rf4.fit(X4_train, Y4_train)
Y4_hat = rf4.predict(X4_test)
cm_rf4 = confusion_matrix(Y4_test, Y4_hat)
cr_rf4 = classification_report(Y4_test, Y4_hat, zero_division = 0)
print(cr_rf4)

              precision    recall  f1-score   support

       False       0.84      0.96      0.90       168
        True       0.96      0.84      0.90       191

    accuracy                           0.90       359
   macro avg       0.90      0.90      0.90       359
weighted avg       0.91      0.90      0.90       359



In [ ]:
# Promising results for dat4
# Now let's tune some hyperparams
def objective_rf(params):
  rf = RandomForestClassifier(**params,n_jobs=-1)
  cv_results = cross_validate(rf, X4, Y4, cv = 3)
  acc = np.mean(cv_results['test_score'])
  return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()

space_rf = {
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.choice('max_features', range(1,200)),
    'n_estimators': hp.choice('n_estimators', range(100,500)),
    'criterion': hp.choice('criterion', ["gini", "entropy"])}

hyperopt.fmin(objective_rf,
              space = space_rf,
              algo = hyperopt.tpe.suggest,
              max_evals=15,
              trials = trials)
# Opt criterion = 0, max_depth = 11, max_features = 11, n_estimators = 297

100%|██████████| 15/15 [12:41<00:00, 50.79s/it, best loss: -0.8967365627841923]


{'criterion': 0, 'max_depth': 11, 'max_features': 11, 'n_estimators': 297}

In [ ]:
def objective_svm(params):
  rf = SVC(**params)
  cv_results = cross_validate(rf, X4, Y4, cv = 3)
  acc = np.mean(cv_results['test_score'])
  return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()

space_svm = {
      'C': hp.choice('C', np.arange(0.005,1.0,0.01)),
      'kernel': hp.choice('kernel',['linear', 'poly', 'rbf']),
      'degree':hp.choice('degree',[2,3,4]),
      'probability':hp.choice('probability',[True])
      }

hyperopt.fmin(objective_svm,
              space = space_svm,
              algo = hyperopt.tpe.suggest,
              max_evals=15,
              trials = trials)
# Opt C = 77, kernel = rbf, prob = 0

100%|██████████| 15/15 [00:32<00:00,  2.17s/it, best loss: -0.9079025749333812]


{'C': 77, 'degree': 0, 'kernel': 2, 'probability': 0}